## FM

In [1]:
import os
import numpy as np
import pandas as pd
from collections import namedtuple

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.python.keras.callbacks import EarlyStopping
from sklearn.preprocessing import  MinMaxScaler, LabelEncoder

from sklearn import metrics
from sklearn.metrics import log_loss
from sklearn.metrics import classification_report, roc_curve

##### 获取数据

In [3]:
def get_criteo_data():
    """ 读取criteo数据集 """
    train_path = '../data/criteo/train.txt'
    test_path = '../data/criteo/test.txt'

    dense_column_names = ['I' + str(i) for i in range(1, 14)]
    sparse_column_names = ['C' + str(i) for i in range(14, 40)]
    column_names = ['label'] + dense_column_names + sparse_column_names

    train_df = pd.read_csv(train_path, names=column_names, sep='\t')
    test_df = pd.read_csv(test_path, names=column_names, sep='\t')
    return train_df, test_df, dense_column_names, sparse_column_names

train_df, test_df, dense_column_names, sparse_column_names = get_criteo_data()
print('train_df.shape: {}, test_df.shape: {}'.format(train_df.shape, test_df.shape))
print('dense_column_names:{}\nsparse_column_names:{} '.format(dense_column_names, sparse_column_names))

train_df.shape: (1000000, 40), test_df.shape: (1000000, 40)
dense_column_names:['I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9', 'I10', 'I11', 'I12', 'I13']
sparse_column_names:['C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26', 'C27', 'C28', 'C29', 'C30', 'C31', 'C32', 'C33', 'C34', 'C35', 'C36', 'C37', 'C38', 'C39'] 


##### 数据预处理

In [6]:
data = pd.concat([train_df, test_df], axis=0)

def label_encoder(y, frequency_threshold=100):
    """ 字典编码，eg: label_encoder([2,2,3,3,3,5,6], 2) => array([2, 2, 1, 1, 1, 0, 0], dtype=int8)
    :params y 类别特征
    :params frequency_threshold 阈值，低于阈值的类别编码为0
    :return: 编码后的numpy数组
    """
    value_counts = pd.value_counts(y)
    categories = value_counts[value_counts >= frequency_threshold].index.to_numpy()
    #print('categories', categories)
    return pd.Categorical(y, categories=categories).codes+1

def data_processing(df, dense_column_names, sparse_column_names):
    df[dense_column_names] = df[dense_column_names].fillna(0.0)
    for f in dense_column_names:
        df[f] = df[f].apply(lambda x: np.log(x+1) if x > -1 else -1)
    
    df[sparse_column_names] = df[sparse_column_names].fillna("-1")
    for f in sparse_column_names:
        df[f] = label_encoder(df[f])
#         lbe = LabelEncoder()
#         df[f] = lbe.fit_transform(df[f])
    return df[dense_column_names + sparse_column_names]

df = data_processing(data, dense_column_names, sparse_column_names)
df['label'] = data['label']

train_df, test_df = df.iloc[0:train_df.shape[0]], df.iloc[train_df.shape[0]:]
print('train_df.shape:{}, test_df.shape:{}'.format(train_df.shape, test_df.shape))

train_df.shape:(1000000, 40), test_df.shape:(1000000, 40)


##### 模型构建

In [10]:
SparseFeature = namedtuple('SparseFeature', ['name', 'vocabulary_size', 'embedding_size'])
DenseFeature = namedtuple('DenseFeature', ['name', 'dimension'])

feature_columns = [SparseFeature(f, vocabulary_size=df[f].nunique(), embedding_size=4) for f in sparse_column_names] + \
[DenseFeature(f, 1) for f in dense_column_names]
feature_columns

[SparseFeature(name='C14', vocabulary_size=138, embedding_size=4),
 SparseFeature(name='C15', vocabulary_size=421, embedding_size=4),
 SparseFeature(name='C16', vocabulary_size=1308, embedding_size=4),
 SparseFeature(name='C17', vocabulary_size=1708, embedding_size=4),
 SparseFeature(name='C18', vocabulary_size=52, embedding_size=4),
 SparseFeature(name='C19', vocabulary_size=10, embedding_size=4),
 SparseFeature(name='C20', vocabulary_size=3251, embedding_size=4),
 SparseFeature(name='C21', vocabulary_size=82, embedding_size=4),
 SparseFeature(name='C22', vocabulary_size=3, embedding_size=4),
 SparseFeature(name='C23', vocabulary_size=2355, embedding_size=4),
 SparseFeature(name='C24', vocabulary_size=2308, embedding_size=4),
 SparseFeature(name='C25', vocabulary_size=1347, embedding_size=4),
 SparseFeature(name='C26', vocabulary_size=1979, embedding_size=4),
 SparseFeature(name='C27', vocabulary_size=26, embedding_size=4),
 SparseFeature(name='C28', vocabulary_size=1960, embedding_si

In [11]:
def model_metric(prob, label, thr=0.5):
    """ 模型评估 """
    # AUC
    fpr, tpr, threshold = metrics.roc_curve(label, prob)
    auc = metrics.auc(fpr, tpr)
    score = metrics.accuracy_score(label, prob > thr)
    # LogLoss
    logloss = log_loss(label, prob)
    print('模型准确率:{}, AUC得分:{}, LogLoss:{}'.format(score, auc, logloss))
    print(classification_report(label, prob > thr, digits=2))
    print('==========================================================')

class FM_Layer(Layer):
    def __init__(self):
        super(FM_Layer, self).__init__()
        
    def call(self, inputs):
        concat_embed_values = inputs
        print('concat_embed_values.shape: ', concat_embed_values.shape) # (None, 26, 4)
        sum_square = tf.square(tf.reduce_sum(concat_embed_values, axis=1, keepdims=True)) # (None, 1, 4)
        print('sum_square.shape: ', sum_square.shape)
        square_sum = tf.reduce_sum(concat_embed_values * concat_embed_values, axis=1, keepdims=True) # (None, 1, 4)
        print('square_sum.shape: ', square_sum.shape)
        output = sum_square - square_sum # 和的平方-平方的和
        output = 0.5 * tf.reduce_sum(output, axis=2, keepdims=False) # (None 1)
        print('output.shape: ', output.shape)
        return output
    
    def compute_output_shape(self, input_shape):
        return (None, 1)
    
def build_input_layers(feature_columns):
    """ 构建输入层 """
    dense_input_dict, sparse_input_dict = {}, {}
    for f in feature_columns:
        if isinstance(f, DenseFeature):
            dense_input_dict[f.name] = Input(shape=(f.dimension, ), name=f.name)
        elif isinstance(f, SparseFeature):
            sparse_input_dict[f.name] = Input(shape=(1, ), name=f.name)
    return dense_input_dict, sparse_input_dict
    
def build_embedding_layers(feature_columns, is_linear):
    """ 构建embedding层 """
    embedding_layer_dict = {}
    sparse_feature_columns = list(filter(lambda x: isinstance(x, SparseFeature), feature_columns))
    if is_linear:
        for f in sparse_feature_columns:
            embedding_layer_dict[f.name] = Embedding(f.vocabulary_size+1, 1, name='1d_emb_' + f.name)
    else:
        for f in sparse_feature_columns:
            embedding_layer_dict[f.name] = Embedding(f.vocabulary_size+1, f.embedding_size, name='kd_emb_' + f.name)
    return embedding_layer_dict
    
def get_linear_logits(dense_input_dict, sparse_input_dict, feature_columns):
    """ 数值特征拼接一起传入全连接层 + 类别特征onehot，flatten，add """
    concat_dense_inputs = Concatenate(axis=1)(list(dense_input_dict.values()))
    dense_logits_output = Dense(1)(concat_dense_inputs)
    
    sparse_feature_columns = list(filter(lambda x: isinstance(x, SparseFeature), feature_columns))
    embedding_layer_dict = build_embedding_layers(feature_columns, is_linear=True)
    
    # embedding(input)查表操作，返回对应input的嵌入向量
    sparse_1d_embed_list = []
    for f in sparse_feature_columns:
        _input = sparse_input_dict[f.name]
        _embed = Flatten()(embedding_layer_dict[f.name](_input))
        sparse_1d_embed_list.append(_embed)
    
    sparse_logits_output = Add()(sparse_1d_embed_list)
    linear_logits = Add()([dense_logits_output, sparse_logits_output])
    return linear_logits
    
def get_fm_logits(sparse_input_dict, kd_embedding_layer_dict, feature_columns):
    """ 取出input所对应的嵌入向量拼接在一起，计算和的平和-平方的和 """
    sparse_feature_columns = list(filter(lambda x: isinstance(x, SparseFeature), feature_columns))
    sparse_kd_embed_list = []
    for f in sparse_feature_columns:
        _input = sparse_input_dict[f.name]
        _embed = kd_embedding_layer_dict[f.name](_input)
        sparse_kd_embed_list.append(_embed)
    
    concat_sparse_kd_embed_list = Concatenate(axis=1)(sparse_kd_embed_list)
    fm_logits = FM_Layer()(concat_sparse_kd_embed_list)
    return fm_logits
    
def FM(feature_columns):
    """ Instantiates FM architecture
    :param feature_columns 
    :param seed
    :param l2_reg L2regularization
    :return: A kears model instance
    """
    dense_input_dict, sparse_input_dict = build_input_layers(feature_columns)
    input_list = list(dense_input_dict.values()) + list(sparse_input_dict.values())
    
    # w * x
    linear_logits = get_linear_logits(dense_input_dict, sparse_input_dict, feature_columns)
    
    # 0.5 * [sum(vixi)**2 - sum(vixi*vixi)
    kd_embedding_layer_dict = build_embedding_layers(feature_columns, is_linear=False)
    fm_logits = get_fm_logits(sparse_input_dict, kd_embedding_layer_dict, feature_columns)
    
    output_logits = Add()([linear_logits, fm_logits])
    output_layer = Activation("sigmoid")(output_logits)
    model = Model(input_list, output_layer)
    return model

model = FM(feature_columns)
model.summary()

concat_embed_values.shape:  (None, 26, 4)
sum_square.shape:  (None, 1, 4)
square_sum.shape:  (None, 1, 4)
output.shape:  (None, 1)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
C14 (InputLayer)                [(None, 1)]          0                                            
__________________________________________________________________________________________________
C15 (InputLayer)                [(None, 1)]          0                                            
__________________________________________________________________________________________________
C16 (InputLayer)                [(None, 1)]          0                                            
__________________________________________________________________________________________________
C17 (InputLayer)                [(None, 1)]          0        

In [12]:
# 模型训练
train_input = {f: train_df[f] for f in dense_column_names + sparse_column_names}
test_input = {f: test_df[f] for f in dense_column_names + sparse_column_names}

my_callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, verbose=2, mode='auto')
]
model.compile('adam',
              loss='binary_crossentropy',
              metrics=["binary_crossentropy", tf.keras.metrics.AUC(name='auc')])
model.fit(train_input,
          train_df['label'].values,
          batch_size=1024,
          epochs=2,
          validation_split=0.2,
          callbacks=my_callbacks)

Epoch 1/2
concat_embed_values.shape:  (None, 26, 4)
sum_square.shape:  (None, 1, 4)
square_sum.shape:  (None, 1, 4)
output.shape:  (None, 1)
concat_embed_values.shape:  (None, 26, 4)
sum_square.shape:  (None, 1, 4)
square_sum.shape:  (None, 1, 4)
output.shape:  (None, 1)
771/782 [============================>.] - ETA: 0s - loss: 0.5795 - binary_crossentropy: 0.5795 - auc: 0.6794concat_embed_values.shape:  (None, 26, 4)
sum_square.shape:  (None, 1, 4)
square_sum.shape:  (None, 1, 4)
output.shape:  (None, 1)
782/782 [==============================] - 8s 5ms/step - loss: 0.5784 - binary_crossentropy: 0.5784 - auc: 0.6802 - val_loss: 0.4861 - val_binary_crossentropy: 0.4861 - val_auc: 0.7527
Epoch 2/2
782/782 [==============================] - 4s 5ms/step - loss: 0.4719 - binary_crossentropy: 0.4719 - auc: 0.7741 - val_loss: 0.4690 - val_binary_crossentropy: 0.4690 - val_auc: 0.7729


In [13]:
# 模型预测与评估
result = model.predict(test_input)
model_metric(np.array([i[0] for i in result]), test_df['label'].values)

concat_embed_values.shape:  (32, 26, 4)
sum_square.shape:  (32, 1, 4)
square_sum.shape:  (32, 1, 4)
output.shape:  (32, 1)
模型准确率:0.782787, AUC得分:0.7702113361598304, LogLoss:0.4685569820745593
              precision    recall  f1-score   support

           0       0.81      0.94      0.87    751819
           1       0.62      0.32      0.42    248181

    accuracy                           0.78   1000000
   macro avg       0.71      0.63      0.64   1000000
weighted avg       0.76      0.78      0.76   1000000

